In [1]:
import pandas as pd

# CSV original com elenco
df_csv = pd.read_csv("movies.csv", sep='|', na_values='\\N')

# JSON convertido para CSV da API TMDB
df_tmdb = pd.read_csv("filmes_guerra_detalhes.csv")


In [2]:
# Exemplo de IDs únicos em cada base
print("IDs no CSV:", df_csv['id'].nunique())
print("IMDb IDs no TMDB:", df_tmdb['imdb_id'].nunique())

# Verificar interseção
ids_csv = set(df_csv['id'].dropna().unique())
ids_tmdb = set(df_tmdb['imdb_id'].dropna().unique())
ids_comuns = ids_csv.intersection(ids_tmdb)

print(f"IDs em comum entre CSV e TMDB: {len(ids_comuns)}")


IDs no CSV: 244543
IMDb IDs no TMDB: 574
IDs em comum entre CSV e TMDB: 511


In [3]:
# Set com os IMDb IDs que estão no TMDB, mas não no CSV
ids_tmdb_nao_estao_csv = ids_tmdb.difference(ids_csv)

print(f"Total de filmes TMDB que não estão no CSV: {len(ids_tmdb_nao_estao_csv)}")

# Ver os títulos desses filmes
df_tmdb_nao_casados = df_tmdb[df_tmdb['imdb_id'].isin(ids_tmdb_nao_estao_csv)][['imdb_id', 'title', 'release_date']]
print(df_tmdb_nao_casados.head(10))


Total de filmes TMDB que não estão no CSV: 63
       imdb_id              title release_date
0   tt31434639    Tempo de Guerra   2025-04-09
1   tt27118357      Major Imortal   2024-10-31
3   tt23872640       Dirty Angels   2024-12-11
6   tt17279496       Guerra Civil   2024-04-10
8   tt19864802      Zona de Risco   2024-02-09
12   tt5177120  Guerra Sem Regras   2024-04-18
15  tt13287846           Napoleão   2023-11-22
16   tt4873118            O Pacto   2023-04-19
32   tt7160372  Zona de Interesse   2023-12-15
39  tt23782584          Número 24   2024-10-30


In [4]:
# Conjunto de imdb_ids do TMDB que não estão no CSV
ids_tmdb_nao_estao_csv = ids_tmdb.difference(ids_csv)

# Filtrar no dataframe do TMDB
df_tmdb_nao_casados = df_tmdb[df_tmdb['imdb_id'].isin(ids_tmdb_nao_estao_csv)][['imdb_id', 'title', 'release_date']]

# Exibir todos os filmes não casados
pd.set_option('display.max_rows', None)  # Mostra todas as linhas
print(df_tmdb_nao_casados.sort_values(by='release_date'))


        imdb_id                                        title release_date
544   tt0059894                             O Jogo da Guerra   1966-04-13
319   tt0078753                            Adeus à Inocência   1979-11-14
136   tt0090163                           Catástrofe Nuclear   1984-09-23
216   tt0092978                              Fuga de Sobibor   1987-04-12
218   tt0114745                                Prova de Fogo   1995-08-25
366   tt0135706                    Quando os Bravos se Calam   1998-06-27
347   tt0266425                                  Conspiração   2001-05-19
497   tt0287535                            O Último Batalhão   2001-12-02
504   tt0317910                        Sob a Névoa da Guerra   2003-10-26
566   tt0440846                                   Fallen Art   2004-09-23
413   tt0821638         Enterrem Meu Coração na Curva do Rio   2007-05-27
420   tt1192431                               Мы из будущего   2008-02-21
272   tt1185616                       

In [5]:
# 1️⃣ Parte comum (interseção)
df_csv_tmdb = df_csv[
    (df_csv['id'].isin(ids_comuns)) &
    (df_csv['anoLancamento'] >= 1950) &
    (df_csv['notaMedia'] >= 6) &
    (df_csv['numeroVotos'] >= 100)
]

df_tmdb_comuns = df_tmdb[df_tmdb['imdb_id'].isin(ids_comuns)]

df_merged_comuns = df_csv_tmdb.merge(
    df_tmdb_comuns,
    how='left',
    left_on='id',
    right_on='imdb_id',
    suffixes=('_csv', '_tmdb')
)

# 2️⃣ Parte exclusiva do CSV (filmes válidos, mas não estão no TMDB)
ids_exclusivos_csv = ids_csv.difference(ids_tmdb)

df_csv_exclusivo = df_csv[
    (df_csv['id'].isin(ids_exclusivos_csv)) &
    (df_csv['anoLancamento'] >= 1950) &
    (df_csv['notaMedia'] >= 6) &
    (df_csv['numeroVotos'] >= 100)
]

# 3️⃣ Parte exclusiva do TMDB (filmes válidos, mas não estão no CSV)
ids_exclusivos_tmdb = ids_tmdb.difference(ids_csv)

df_tmdb_exclusivo = df_tmdb[
    (df_tmdb['imdb_id'].isin(ids_exclusivos_tmdb)) &
    (df_tmdb['release_date'] >= '1950-01-01') &
    (df_tmdb['vote_average'] >= 6) &
    (df_tmdb['vote_count'] >= 100)
]


In [7]:



# ⚠️ Garanta que esses DataFrames já existem antes de rodar isso:
# - df_csv_exclusivo
# - df_tmdb_exclusivo
# - df_tmdb
# - df_csv

# 1️⃣ Forçar cópias para evitar o SettingWithCopyWarning
df_csv_exclusivo = df_csv_exclusivo.copy()
df_tmdb_exclusivo = df_tmdb_exclusivo.copy()

# 2️⃣ Adicionar colunas da TMDB no CSV exclusivo (preenchidas com pd.NA)
for col in df_tmdb.columns:
    if col not in df_csv_exclusivo.columns:
        df_csv_exclusivo.loc[:, col] = pd.NA

# 3️⃣ Adicionar colunas do CSV no TMDB exclusivo (preenchidas com pd.NA)
for col in df_csv.columns:
    if col not in df_tmdb_exclusivo.columns:
        df_tmdb_exclusivo.loc[:, col] = pd.NA

# 4️⃣ Padronizar a chave 'id' e 'imdb_id'
df_csv_exclusivo.loc[:, 'imdb_id'] = df_csv_exclusivo['id']
df_tmdb_exclusivo.loc[:, 'id'] = df_tmdb_exclusivo['imdb_id']



In [8]:
# 1️⃣ Criar uma flag de origem para cada parte
df_merged_comuns = df_merged_comuns.copy()
df_csv_exclusivo = df_csv_exclusivo.copy()
df_tmdb_exclusivo = df_tmdb_exclusivo.copy()

df_merged_comuns["fonte_origem"] = "csv+tmdb"
df_csv_exclusivo["fonte_origem"] = "csv"
df_tmdb_exclusivo["fonte_origem"] = "tmdb"

# 2️⃣ Garantir mesmas colunas em todas as bases
colunas_comuns = df_merged_comuns.columns.intersection(df_csv_exclusivo.columns)
colunas_comuns = colunas_comuns.intersection(df_tmdb_exclusivo.columns)

# 3️⃣ Concatenar as 3 partes em um DataFrame final
df_final = pd.concat([
    df_merged_comuns[colunas_comuns],
    df_csv_exclusivo[colunas_comuns],
    df_tmdb_exclusivo[colunas_comuns]
], ignore_index=True)


C:\Users\luiza\AppData\Local\Temp\ipykernel_21668\3946869102.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([


In [9]:
# Quantidade total de filmes únicos
print("Total de filmes únicos:", df_final['id'].nunique())

# Contagem por origem
print(df_final['fonte_origem'].value_counts())


KeyError: 'id'

In [10]:
print(df_final.columns.tolist())


['tituloPincipal', 'tituloOriginal', 'anoLancamento', 'tempoMinutos', 'genero', 'notaMedia', 'numeroVotos', 'generoArtista', 'personagem', 'nomeArtista', 'anoNascimento', 'anoFalecimento', 'profissao', 'titulosMaisConhecidos', 'adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'imdb_id', 'origin_country', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'credits.cast', 'credits.crew', 'keywords.keywords', 'release_dates.results', 'belongs_to_collection.id', 'belongs_to_collection.name', 'belongs_to_collection.poster_path', 'belongs_to_collection.backdrop_path', 'fonte_origem']


In [11]:
# Quantidade total de filmes únicos
print("Total de filmes únicos:", df_final['imdb_id'].nunique())

# Contagem por origem
print(df_final['fonte_origem'].value_counts())


Total de filmes únicos: 52538
fonte_origem
csv         211330
csv+tmdb      1942
tmdb            63
Name: count, dtype: int64


In [12]:
# Filtro para filmes válidos do CSV
filtro_csv = (
    (df_final['fonte_origem'].isin(['csv', 'csv+tmdb'])) &
    (df_final['anoLancamento'].notnull()) &
    (df_final['notaMedia'] >= 6) &
    (df_final['numeroVotos'] >= 100) &
    (df_final['anoLancamento'] >= 1950)
)

# Filtro para filmes válidos do TMDB (filtrados na ingestão, mas vamos reforçar)
filtro_tmdb = (
    (df_final['fonte_origem'].isin(['tmdb', 'csv+tmdb'])) &
    (df_final['vote_average'] >= 6) &
    (df_final['vote_count'] >= 100) &
    (pd.to_datetime(df_final['release_date'], errors='coerce').dt.year >= 1950)
)

# Filmes que atendem aos critérios em qualquer uma das fontes
filtro_geral = filtro_csv | filtro_tmdb

# Aplicar filtro final
df_filmes_filtrados = df_final[filtro_geral]

# Ver resultados
print("🔎 Total de registros após aplicar os filtros:", len(df_filmes_filtrados))
print("🎬 Total de filmes únicos:", df_filmes_filtrados['imdb_id'].nunique())
print("📁 Distribuição por fonte:")
print(df_filmes_filtrados['fonte_origem'].value_counts())


🔎 Total de registros após aplicar os filtros: 213335
🎬 Total de filmes únicos: 52538
📁 Distribuição por fonte:
fonte_origem
csv         211330
csv+tmdb      1942
tmdb            63
Name: count, dtype: int64


vou trabalhar com os filmes que tenham apenas no csv e tmdb (1942)

so filmes de guerra,  a base é 1911

In [13]:
import ast

# Função para verificar se 'guerra' está nos gêneros do TMDB
def contem_genero_guerra_tmdb(generos_str):
    try:
        generos = ast.literal_eval(generos_str)
        return any('guerra' in g['name'].lower() for g in generos)
    except:
        return False

# Filtro por gênero 'guerra' no CSV
filtro_guerra_csv = df_filmes_filtrados['genero'].str.contains('War', case=False, na=False)

# Filtro por gênero 'guerra' no TMDB
filtro_guerra_tmdb = df_filmes_filtrados['genres'].apply(contem_genero_guerra_tmdb)

# Combinar os dois filtros
filtro_guerra_total = filtro_guerra_csv | filtro_guerra_tmdb

# Aplicar o filtro
df_filmes_guerra = df_filmes_filtrados[filtro_guerra_total].copy()

# Exibir resultados
print("🎯 Total de filmes de guerra (filtrados):", df_filmes_guerra['imdb_id'].nunique())
print("📁 Distribuição por fonte:")
print(df_filmes_guerra['fonte_origem'].value_counts())


🎯 Total de filmes de guerra (filtrados): 1911
📁 Distribuição por fonte:
fonte_origem
csv         5417
csv+tmdb    1942
tmdb          63
Name: count, dtype: int64


In [14]:
df_guerra_completo = df_filmes_guerra[df_filmes_guerra['fonte_origem'] == 'csv+tmdb']
print("🎯 Total de filmes de guerra com dados CSV + TMDB:", df_guerra_completo['imdb_id'].nunique())


🎯 Total de filmes de guerra com dados CSV + TMDB: 486


tratando dados

In [15]:
df_refined = df_guerra_completo.copy()


In [18]:
df_refined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1942 entries, 0 to 1941
Data columns (total 48 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   tituloPincipal                       1942 non-null   object 
 1   tituloOriginal                       1942 non-null   object 
 2   anoLancamento                        1942 non-null   Int64  
 3   tempoMinutos                         1942 non-null   Int64  
 4   genero                               1942 non-null   object 
 5   notaMedia                            1942 non-null   float64
 6   numeroVotos                          1942 non-null   Int64  
 7   generoArtista                        1942 non-null   object 
 8   personagem                           1929 non-null   object 
 9   nomeArtista                          1942 non-null   object 
 10  anoNascimento                        1758 non-null   float64
 11  anoFalecimento                     

In [16]:
# Garantir que anoLancamento seja inteiro
df_refined['anoLancamento'] = df_refined['anoLancamento'].astype('Int64')

# Garantir que tempoMinutos seja inteiro (pode ter valores nulos)
df_refined['tempoMinutos'] = df_refined['tempoMinutos'].astype('Int64')

# Nota média e número de votos do CSV
df_refined['notaMedia'] = df_refined['notaMedia'].astype(float)
df_refined['numeroVotos'] = df_refined['numeroVotos'].astype('Int64')

# vote_average e vote_count do TMDB
df_refined['vote_average'] = df_refined['vote_average'].astype(float)
df_refined['vote_count'] = df_refined['vote_count'].astype('Int64')


In [19]:
df_refined['release_date'] = pd.to_datetime(df_refined['release_date'], errors='coerce')
df_refined['ano_lancamento_tmdb'] = df_refined['release_date'].dt.year.astype('Int64')


In [20]:
import ast

def parse_json_column(col):
    return col.apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

colunas_json = [
    'genres', 'keywords.keywords', 'credits.cast', 'credits.crew',
    'production_companies', 'production_countries', 'release_dates.results'
]

for col in colunas_json:
    if col in df_refined.columns:
        df_refined[col] = parse_json_column(df_refined[col])


In [21]:
# Verificar colunas críticas
print(df_refined[['imdb_id', 'tituloOriginal', 'anoLancamento', 'notaMedia', 'vote_average', 'release_date']].isnull().sum())


imdb_id           0
tituloOriginal    0
anoLancamento     0
notaMedia         0
vote_average      0
release_date      0
dtype: int64


In [24]:
# Ver informações gerais (colunas, tipos e valores ausentes)
df_refined.info()

# Identificar colunas que não possuem listas ou dicionários (evita erro no nunique)
colunas_sem_estruturas_complexas = [
    col for col in df_refined.columns
    if df_refined[col].apply(lambda x: isinstance(x, (list, dict))).sum() == 0
]

# Mostrar as colunas com maior número de valores únicos (entre as simples)
print("\n🔍 Principais colunas com valores únicos:")
print(df_refined[colunas_sem_estruturas_complexas].nunique().sort_values(ascending=False).head(15))

# Exibir amostra dos dados
print("\n🧪 Amostra dos dados:")
display(df_refined[colunas_sem_estruturas_complexas].head(3))


<class 'pandas.core.frame.DataFrame'>
Index: 1942 entries, 0 to 1941
Data columns (total 49 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   tituloPincipal                       1942 non-null   object        
 1   tituloOriginal                       1942 non-null   object        
 2   anoLancamento                        1942 non-null   Int64         
 3   tempoMinutos                         1942 non-null   Int64         
 4   genero                               1942 non-null   object        
 5   notaMedia                            1942 non-null   float64       
 6   numeroVotos                          1942 non-null   Int64         
 7   generoArtista                        1942 non-null   object        
 8   personagem                           1929 non-null   object        
 9   nomeArtista                          1942 non-null   object        
 10  anoNascimento    

,tituloPincipal,tituloOriginal,anoLancamento,tempoMinutos,genero,notaMedia,numeroVotos,generoArtista,personagem,nomeArtista,...,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,fonte_origem,ano_lancamento_tmdb
0,Captain Horatio Hornblower,Captain Horatio Hornblower R.N.,1951,117,"Action,Adventure,Drama",7.3,6851,actor,Capt. Horatio Hornblower R.N,Gregory Peck,...,Falcão dos Mares,False,6.9,124,NaN,NaN,NaN,NaN,csv+tmdb,1951
1,Captain Horatio Hornblower,Captain Horatio Hornblower R.N.,1951,117,"Action,Adventure,Drama",7.3,6851,actress,Lady Barbara Wellesley,Virginia Mayo,...,Falcão dos Mares,False,6.9,124,NaN,NaN,NaN,NaN,csv+tmdb,1951
2,Captain Horatio Hornblower,Captain Horatio Hornblower R.N.,1951,117,"Action,Adventure,Drama",7.3,6851,actor,Lt. William Bush,Robert Beatty,...,Falcão dos Mares,False,6.9,124,NaN,NaN,NaN,NaN,csv+tmdb,1951


criando a tabela fato

In [29]:
# Copiar o DataFrame base
df_fato_filmes = df_refined.copy()

# Criar colunas derivadas
df_fato_filmes['id_filme'] = df_fato_filmes['imdb_id']
df_fato_filmes['decada'] = df_fato_filmes['anoLancamento'] // 10 * 10

# Normalizar colunas de gênero
# Se "genres" da TMDB estiver disponível, usa ele; senão, usa o do CSV
def extrair_generos(row):
    try:
        if isinstance(row['genres'], str) and row['genres'].startswith('['):
            # formato da API: "[{'id': ..., 'name': 'War'}]"
            import ast
            gen_list = ast.literal_eval(row['genres'])
            return ','.join([g['name'] for g in gen_list if 'name' in g])
        elif pd.notnull(row['genero']):
            return row['genero']
        else:
            return pd.NA
    except:
        return pd.NA

df_fato_filmes['generos'] = df_fato_filmes.apply(extrair_generos, axis=1)

# Padronizar duração
def obter_duracao(row):
    if pd.notnull(row['tempoMinutos']):
        return row['tempoMinutos']
    try:
        return int(row['runtime'])
    except:
        return pd.NA

df_fato_filmes['duracao'] = df_fato_filmes.apply(obter_duracao, axis=1)

# Selecionar e renomear as colunas da fato
fato_filmes = df_fato_filmes[[
    'id_filme',
    'tituloPincipal',
    'anoLancamento',
    'decada',
    'generos',
    'notaMedia',
    'numeroVotos',
    'budget',
    'revenue',
    'duracao',
    'original_language',
    'fonte_origem'
]].rename(columns={
    'tituloPincipal': 'titulo',
    'anoLancamento': 'ano_lancamento',
    'notaMedia': 'nota_media',
    'numeroVotos': 'numero_votos',
    'budget': 'orcamento',
    'revenue': 'receita',
    'original_language': 'idioma_original'
})

# Verificar o resultado
print("✅ Tabela fato_filmes gerada com sucesso!")
display(fato_filmes.head(3))
print("\n📏 Formato:", fato_filmes.shape)
print("🧩 Valores ausentes por coluna:")
display(fato_filmes.isnull().sum())


✅ Tabela fato_filmes gerada com sucesso!


,id_filme,titulo,ano_lancamento,decada,generos,nota_media,numero_votos,orcamento,receita,duracao,idioma_original,fonte_origem
0,tt0043379,Captain Horatio Hornblower,1951,1950,"Action,Adventure,Drama",7.3,6851,0,0,117,en,csv+tmdb
1,tt0043379,Captain Horatio Hornblower,1951,1950,"Action,Adventure,Drama",7.3,6851,0,0,117,en,csv+tmdb
2,tt0043379,Captain Horatio Hornblower,1951,1950,"Action,Adventure,Drama",7.3,6851,0,0,117,en,csv+tmdb



📏 Formato: (1942, 12)
🧩 Valores ausentes por coluna:


id_filme           0
titulo             0
ano_lancamento     0
decada             0
generos            0
nota_media         0
numero_votos       0
orcamento          0
receita            0
duracao            0
idioma_original    0
fonte_origem       0
dtype: int64

tabela fato final (com valores unicos)

In [26]:
fato_filmes_final = fato_filmes.drop_duplicates(subset='id_filme')
print("🎬 Total de filmes únicos na tabela fato:", fato_filmes_final.shape[0])


🎬 Total de filmes únicos na tabela fato: 486


vendo p orcamento e receita diferente de zero

In [30]:
# Garantir que estamos trabalhando com uma cópia segura do dataframe tratado
df_base = df_refined.copy()

# ✅ Conversões seguras para orçamento e receita (usando NaN em vez de 0 para ausentes)
df_base['orcamento'] = pd.to_numeric(df_base['budget'], errors='coerce')
df_base['receita'] = pd.to_numeric(df_base['revenue'], errors='coerce')

# ✅ Geração da coluna 'decada'
df_base['decada'] = (df_base['ano_lancamento'] // 10) * 10

# ✅ Construção da tabela fato
fato_filmes = pd.DataFrame({
    'id_filme': df_base['imdb_id'],
    'titulo': df_base['tituloPincipal'],
    'ano_lancamento': df_base['ano_lancamento'],
    'decada': df_base['decada'],
    'generos': df_base['genero'],
    'nota_media': df_base['notaMedia'],
    'numero_votos': df_base['numeroVotos'],
    'orcamento': df_base['orcamento'],
    'receita': df_base['receita'],
    'duracao': df_base['tempoMinutos'],
    'idioma_original': df_base['original_language'],
    'fonte_origem': df_base['fonte_origem']
})

# ✅ Remover duplicatas (uma linha por filme)
fato_filmes_final = fato_filmes.drop_duplicates(subset='id_filme')

# ✅ Verificações
print("🎬 Total de filmes únicos na tabela fato:", fato_filmes_final.shape[0])
print("\n📏 Formato final:", fato_filmes_final.shape)
print("\n🧩 Valores ausentes por coluna:")
print(fato_filmes_final.isnull().sum())
print("\n🧪 Amostra:")
display(fato_filmes_final.head())


KeyError: 'ano_lancamento'

In [31]:
import pandas as pd
import ast

# Copiar o DataFrame base
df_fato_filmes = df_refined.copy()

# Criar colunas derivadas
df_fato_filmes['id_filme'] = df_fato_filmes['imdb_id']
df_fato_filmes['decada'] = (df_fato_filmes['anoLancamento'] // 10) * 10

# Normalizar colunas de gênero
def extrair_generos(row):
    try:
        if isinstance(row['genres'], str) and row['genres'].startswith('['):
            # Formato API: lista de dicts, extrair nome dos gêneros
            gen_list = ast.literal_eval(row['genres'])
            return ','.join([g['name'] for g in gen_list if 'name' in g])
        elif pd.notnull(row['genero']):
            return row['genero']
        else:
            return pd.NA
    except:
        return pd.NA

df_fato_filmes['generos'] = df_fato_filmes.apply(extrair_generos, axis=1)

# Padronizar duração
def obter_duracao(row):
    if pd.notnull(row['tempoMinutos']):
        return row['tempoMinutos']
    try:
        return int(row['runtime'])
    except:
        return pd.NA

df_fato_filmes['duracao'] = df_fato_filmes.apply(obter_duracao, axis=1)

# Selecionar e renomear as colunas para a tabela fato
fato_filmes = df_fato_filmes[[
    'id_filme',
    'tituloPincipal',
    'anoLancamento',
    'decada',
    'generos',
    'notaMedia',
    'numeroVotos',
    'budget',
    'revenue',
    'duracao',
    'original_language',
    'fonte_origem'
]].rename(columns={
    'tituloPincipal': 'titulo',
    'anoLancamento': 'ano_lancamento',
    'notaMedia': 'nota_media',
    'numeroVotos': 'numero_votos',
    'budget': 'orcamento',
    'revenue': 'receita',
    'original_language': 'idioma_original'
})

# Verificar o resultado
print("✅ Tabela fato_filmes gerada com sucesso!")
display(fato_filmes.head(3))
print("\n📏 Formato:", fato_filmes.shape)
print("🧩 Valores ausentes por coluna:")
display(fato_filmes.isnull().sum())


✅ Tabela fato_filmes gerada com sucesso!


,id_filme,titulo,ano_lancamento,decada,generos,nota_media,numero_votos,orcamento,receita,duracao,idioma_original,fonte_origem
0,tt0043379,Captain Horatio Hornblower,1951,1950,"Action,Adventure,Drama",7.3,6851,0,0,117,en,csv+tmdb
1,tt0043379,Captain Horatio Hornblower,1951,1950,"Action,Adventure,Drama",7.3,6851,0,0,117,en,csv+tmdb
2,tt0043379,Captain Horatio Hornblower,1951,1950,"Action,Adventure,Drama",7.3,6851,0,0,117,en,csv+tmdb



📏 Formato: (1942, 12)
🧩 Valores ausentes por coluna:


id_filme           0
titulo             0
ano_lancamento     0
decada             0
generos            0
nota_media         0
numero_votos       0
orcamento          0
receita            0
duracao            0
idioma_original    0
fonte_origem       0
dtype: int64

orcamento e receita diferente de zero

In [32]:
import pandas as pd
import ast

# Copiar o DataFrame base
df_fato_filmes = df_refined.copy()

# Criar colunas derivadas
df_fato_filmes['id_filme'] = df_fato_filmes['imdb_id']
df_fato_filmes['decada'] = (df_fato_filmes['anoLancamento'] // 10) * 10

# Normalizar colunas de gênero
def extrair_generos(row):
    try:
        if isinstance(row['genres'], str) and row['genres'].startswith('['):
            # Formato API: lista de dicts, extrair nome dos gêneros
            gen_list = ast.literal_eval(row['genres'])
            return ','.join([g['name'] for g in gen_list if 'name' in g])
        elif pd.notnull(row['genero']):
            return row['genero']
        else:
            return pd.NA
    except:
        return pd.NA

df_fato_filmes['generos'] = df_fato_filmes.apply(extrair_generos, axis=1)

# Padronizar duração
def obter_duracao(row):
    if pd.notnull(row['tempoMinutos']):
        return row['tempoMinutos']
    try:
        return int(row['runtime'])
    except:
        return pd.NA

df_fato_filmes['duracao'] = df_fato_filmes.apply(obter_duracao, axis=1)

# Converter orçamento e receita para numérico e substituir zeros por NA
df_fato_filmes['orcamento'] = pd.to_numeric(df_fato_filmes['budget'], errors='coerce')
df_fato_filmes['receita'] = pd.to_numeric(df_fato_filmes['revenue'], errors='coerce')

df_fato_filmes.loc[df_fato_filmes['orcamento'] == 0, 'orcamento'] = pd.NA
df_fato_filmes.loc[df_fato_filmes['receita'] == 0, 'receita'] = pd.NA

# Selecionar e renomear as colunas para a tabela fato
fato_filmes = df_fato_filmes[[
    'id_filme',
    'tituloPincipal',
    'anoLancamento',
    'decada',
    'generos',
    'notaMedia',
    'numeroVotos',
    'orcamento',
    'receita',
    'duracao',
    'original_language',
    'fonte_origem'
]].rename(columns={
    'tituloPincipal': 'titulo',
    'anoLancamento': 'ano_lancamento',
    'notaMedia': 'nota_media',
    'numeroVotos': 'numero_votos',
    'original_language': 'idioma_original'
})

# Verificar o resultado
print("✅ Tabela fato_filmes gerada com sucesso!")
display(fato_filmes.head(3))
print("\n📏 Formato:", fato_filmes.shape)
print("🧩 Valores ausentes por coluna:")
display(fato_filmes.isnull().sum())


✅ Tabela fato_filmes gerada com sucesso!


,id_filme,titulo,ano_lancamento,decada,generos,nota_media,numero_votos,orcamento,receita,duracao,idioma_original,fonte_origem
0,tt0043379,Captain Horatio Hornblower,1951,1950,"Action,Adventure,Drama",7.3,6851,NaN,NaN,117,en,csv+tmdb
1,tt0043379,Captain Horatio Hornblower,1951,1950,"Action,Adventure,Drama",7.3,6851,NaN,NaN,117,en,csv+tmdb
2,tt0043379,Captain Horatio Hornblower,1951,1950,"Action,Adventure,Drama",7.3,6851,NaN,NaN,117,en,csv+tmdb



📏 Formato: (1942, 12)
🧩 Valores ausentes por coluna:


id_filme             0
titulo               0
ano_lancamento       0
decada               0
generos              0
nota_media           0
numero_votos         0
orcamento          705
receita            668
duracao              0
idioma_original      0
fonte_origem         0
dtype: int64

In [33]:
# Remover linhas duplicadas por id_filme, mantendo a primeira ocorrência
fato_filmes_final = fato_filmes.drop_duplicates(subset='id_filme').reset_index(drop=True)

print("🎬 Total de filmes únicos na tabela fato:", fato_filmes_final.shape[0])

# Visualizar algumas linhas para conferir
display(fato_filmes_final.head())


🎬 Total de filmes únicos na tabela fato: 486


,id_filme,titulo,ano_lancamento,decada,generos,nota_media,numero_votos,orcamento,receita,duracao,idioma_original,fonte_origem
0,tt0043379,Captain Horatio Hornblower,1951,1950,"Action,Adventure,Drama",7.3,6851,NaN,NaN,117,en,csv+tmdb
1,tt0043461,The Desert Fox: The Story of Rommel,1951,1950,"Biography,Drama,War",6.9,6362,NaN,NaN,88,en,csv+tmdb
2,tt0043686,Forbidden Games,1952,1950,"Comedy,Drama,War",8.0,12414,NaN,10188.0,86,fr,csv+tmdb
3,tt0044602,Fanfan la Tulipe,1952,1950,"Adventure,Comedy,Romance",7.2,3120,NaN,NaN,102,fr,csv+tmdb
4,tt0045793,From Here to Eternity,1953,1950,"Drama,Romance,War",7.6,47555,1650000.0,30500000.0,118,en,csv+tmdb


conferindo os daaddos faltantes e filmes q n temm guerra

In [34]:
# 1. Verificar valores ausentes na tabela fato final
print("🧩 Valores ausentes por coluna na tabela fato final:")
print(fato_filmes_final.isnull().sum())

# 2. Confirmar que todos os filmes possuem o gênero "War" na coluna generos
# Aqui vamos ver quantos filmes não têm "War" no campo generos (caso haja algum erro)
filmes_sem_guerra = fato_filmes_final[~fato_filmes_final['generos'].str.contains('War', case=False, na=False)]

print(f"\n🎯 Filmes sem gênero 'War' na tabela fato final: {filmes_sem_guerra.shape[0]}")

if filmes_sem_guerra.shape[0] > 0:
    print("\nExemplo desses filmes:")
    display(filmes_sem_guerra[['id_filme', 'titulo', 'generos']].head())
else:
    print("✅ Todos os filmes possuem o gênero 'War'.")


🧩 Valores ausentes por coluna na tabela fato final:
id_filme             0
titulo               0
ano_lancamento       0
decada               0
generos              0
nota_media           0
numero_votos         0
orcamento          177
receita            168
duracao              0
idioma_original      0
fonte_origem         0
dtype: int64

🎯 Filmes sem gênero 'War' na tabela fato final: 238

Exemplo desses filmes:


,id_filme,titulo,generos
0,tt0043379,Captain Horatio Hornblower,"Action,Adventure,Drama"
3,tt0044602,Fanfan la Tulipe,"Adventure,Comedy,Romance"
12,tt0049233,Friendly Persuasion,"Drama,Family,Romance"
15,tt0050356,The Enemy Below,"Action,Adventure,Drama"
33,tt0053580,The Alamo,"Adventure,Drama,History"


In [37]:
# 1. Verifique se a string "War" está contida na coluna generos (depois da limpeza que fizemos)
filtro_war_tmdb = df_refined['generos'].str.contains('War', na=False)

# 2. Verifique se a string "War" está presente na coluna genero (do CSV original)
filtro_war_csv = df_refined['genero'].str.contains('War', na=False)

# 3. Casos onde o TMDB considera "War", mas o CSV não
filtro_divergente = filtro_war_tmdb & (~filtro_war_csv)

# 4. Aplicar o filtro e exibir
filmes_divergentes = df_refined[filtro_divergente][['id_filme', 'tituloPincipal', 'genero', 'generos', 'fonte_origem']].drop_duplicates()

print(f"🔍 Total de filmes com gênero 'War' no TMDB mas não no CSV: {filmes_divergentes.shape[0]}")
display(filmes_divergentes.head(10))


KeyError: 'generos'

In [38]:
import ast

def extrair_generos(row):
    try:
        if isinstance(row['genres'], str) and row['genres'].startswith('['):
            gen_list = ast.literal_eval(row['genres'])
            return ','.join([g['name'] for g in gen_list if 'name' in g])
        elif pd.notnull(row['genero']):
            return row['genero']
        else:
            return pd.NA
    except:
        return pd.NA

# Criar a coluna 'generos'
df_refined['generos'] = df_refined.apply(extrair_generos, axis=1)


In [39]:
# Verificação entre TMDB e CSV
filtro_war_tmdb = df_refined['generos'].str.contains('War', na=False)
filtro_war_csv = df_refined['genero'].str.contains('War', na=False)
filtro_divergente = filtro_war_tmdb & (~filtro_war_csv)

# Visualizar divergências
filmes_divergentes = df_refined[filtro_divergente][['id_filme', 'tituloPincipal', 'genero', 'generos', 'fonte_origem']].drop_duplicates()

print(f"🔍 Total de filmes com gênero 'War' no TMDB mas não no CSV: {filmes_divergentes.shape[0]}")
display(filmes_divergentes.head(10))


KeyError: "['id_filme'] not in index"

In [40]:
df_refined['id_filme'] = df_refined['imdb_id']


In [41]:
# Filtros para identificar divergência entre TMDB e CSV
filtro_war_tmdb = df_refined['generos'].str.contains('War', na=False)
filtro_war_csv = df_refined['genero'].str.contains('War', na=False)
filtro_divergente = filtro_war_tmdb & (~filtro_war_csv)

# Visualização dos filmes divergentes
filmes_divergentes = df_refined[filtro_divergente][[
    'id_filme', 'tituloPincipal', 'genero', 'generos', 'fonte_origem'
]].drop_duplicates()

print(f"🔍 Total de filmes com gênero 'War' no TMDB mas não no CSV: {filmes_divergentes.shape[0]}")
display(filmes_divergentes.head(10))


🔍 Total de filmes com gênero 'War' no TMDB mas não no CSV: 0


,id_filme,tituloPincipal,genero,generos,fonte_origem


In [42]:
# Verifica filmes com "War" no CSV mas não no TMDB
filtro_war_csv = df_refined['genero'].str.contains('War', na=False)
filtro_war_tmdb = df_refined['generos'].str.contains('War', na=False)
filtro_divergente_csv = filtro_war_csv & (~filtro_war_tmdb)

filmes_divergentes_csv = df_refined[filtro_divergente_csv][[
    'id_filme', 'tituloPincipal', 'genero', 'generos', 'fonte_origem'
]].drop_duplicates()

print(f"📌 Filmes com 'War' no CSV mas não no TMDB: {filmes_divergentes_csv.shape[0]}")
display(filmes_divergentes_csv.head(10))


📌 Filmes com 'War' no CSV mas não no TMDB: 0


,id_filme,tituloPincipal,genero,generos,fonte_origem


In [43]:
total_filmes_guerra = df_refined['id_filme'].nunique()
print(f"🎬 Total de filmes de guerra únicos na base: {total_filmes_guerra}")


🎬 Total de filmes de guerra únicos na base: 486


In [44]:
filmes_guerra_confirmados = df_refined[df_refined['generos'].str.contains('War', na=False)]
print(f"🎯 Filmes com gênero 'War' confirmado: {filmes_guerra_confirmados['id_filme'].nunique()}")


🎯 Filmes com gênero 'War' confirmado: 248


In [45]:
filmes_guerra_por_fonte = df_refined[['id_filme', 'fonte_origem']].drop_duplicates()
print("📊 Quantidade de filmes de guerra por fonte de origem:")
print(filmes_guerra_por_fonte['fonte_origem'].value_counts())


📊 Quantidade de filmes de guerra por fonte de origem:
fonte_origem
csv+tmdb    486
Name: count, dtype: int64


In [47]:
# Garantir base dos 486 válidos
df_base = df_refined[df_refined['fonte_origem'] == 'csv+tmdb'].copy()

# Filmes onde 'generos' não contém 'War'
filtro_nao_war = ~df_base['generos'].str.contains('War', na=False)

# Aplicar filtro
filmes_suspeitos = df_base[filtro_nao_war].copy()

# Converter colunas problemáticas para string para evitar erro de unhashable
colunas_para_ver = ['id_filme', 'tituloPincipal', 'genero', 'genres', 'generos']
filmes_suspeitos[colunas_para_ver] = filmes_suspeitos[colunas_para_ver].astype(str)

# Exibir resultado
print(f"🎯 Filmes com fonte 'csv+tmdb' mas sem 'War' no campo generos: {filmes_suspeitos.shape[0]}")
display(filmes_suspeitos[colunas_para_ver].drop_duplicates().head(10))



🎯 Filmes com fonte 'csv+tmdb' mas sem 'War' no campo generos: 956


,id_filme,tituloPincipal,genero,genres,generos
0,tt0043379,Captain Horatio Hornblower,"Action,Adventure,Drama","[{'id': 12, 'name': 'Aventura'}, {'id': 10752,...","Action,Adventure,Drama"
12,tt0044602,Fanfan la Tulipe,"Adventure,Comedy,Romance","[{'id': 10749, 'name': 'Romance'}, {'id': 12, ...","Adventure,Comedy,Romance"
48,tt0049233,Friendly Persuasion,"Drama,Family,Romance","[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...","Drama,Family,Romance"
60,tt0050356,The Enemy Below,"Action,Adventure,Drama","[{'id': 10752, 'name': 'Guerra'}]","Action,Adventure,Drama"
130,tt0053580,The Alamo,"Adventure,Drama,History","[{'id': 10752, 'name': 'Guerra'}, {'id': 12, '...","Adventure,Drama,History"
142,tt0054310,Sink the Bismarck!,"Action,Drama,History","[{'id': 10752, 'name': 'Guerra'}, {'id': 18, '...","Action,Drama,History"
146,tt0054331,Spartacus,"Adventure,Biography,Drama","[{'id': 36, 'name': 'História'}, {'id': 10752,...","Adventure,Biography,Drama"
162,tt0054847,El Cid,"Biography,Drama,History","[{'id': 28, 'name': 'Ação'}, {'id': 18, 'name'...","Biography,Drama,History"
166,tt0054953,The Guns of Navarone,"Action,Adventure,Drama","[{'id': 10752, 'name': 'Guerra'}, {'id': 12, '...","Action,Adventure,Drama"
174,tt0055719,The 300 Spartans,"Adventure,Drama,History","[{'id': 12, 'name': 'Aventura'}, {'id': 36, 'n...","Adventure,Drama,History"


In [48]:
import ast

def reconstruir_generos(row):
    try:
        if isinstance(row['genres'], str) and row['genres'].startswith('['):
            lista = ast.literal_eval(row['genres'])
            nomes_ingles = [g['name'] for g in lista if 'name' in g]
            return ','.join(nomes_ingles)
        elif pd.notnull(row['genero']):
            return row['genero']
        else:
            return pd.NA
    except:
        return pd.NA

# Aplicar no df_refined
df_refined['generos'] = df_refined.apply(reconstruir_generos, axis=1)


In [49]:
# Checar se "War" aparece corretamente agora
filtro_war = df_refined['generos'].str.contains('War', na=False)
df_war_final = df_refined[filtro_war]

print("🎯 Total de filmes com 'War' na coluna generos:", df_war_final['id_filme'].nunique())


🎯 Total de filmes com 'War' na coluna generos: 248


In [50]:
import ast

# Mapeamento dos gêneros mais comuns (do TMDB em pt → en)
mapeamento_generos = {
    'Ação': 'Action',
    'Aventura': 'Adventure',
    'Animação': 'Animation',
    'Comédia': 'Comedy',
    'Crime': 'Crime',
    'Documentário': 'Documentary',
    'Drama': 'Drama',
    'Família': 'Family',
    'Fantasia': 'Fantasy',
    'História': 'History',
    'Terror': 'Horror',
    'Música': 'Music',
    'Mistério': 'Mystery',
    'Romance': 'Romance',
    'Ficção científica': 'Science Fiction',
    'Cinema TV': 'TV Movie',
    'Thriller': 'Thriller',
    'Guerra': 'War',
    'Faroeste': 'Western'
}

def reconstruir_generos_traduzido(row):
    try:
        if isinstance(row['genres'], str) and row['genres'].startswith('['):
            lista = ast.literal_eval(row['genres'])
            nomes_traduzidos = [mapeamento_generos.get(g['name'], g['name']) for g in lista if 'name' in g]
            return ','.join(nomes_traduzidos)
        elif pd.notnull(row['genero']):
            return row['genero']
        else:
            return pd.NA
    except:
        return pd.NA

# Atualiza a coluna 'generos' com nomes traduzidos
df_refined['generos'] = df_refined.apply(reconstruir_generos_traduzido, axis=1)


In [51]:
filtro_war = df_refined['generos'].str.contains('War', na=False)
df_war_final = df_refined[filtro_war]

print("🎯 Total de filmes com 'War' na coluna generos (com tradução):", df_war_final['id_filme'].nunique())


🎯 Total de filmes com 'War' na coluna generos (com tradução): 248


In [52]:
# Filmes com 'Guerra' no campo 'genres' bruto do TMDB
tmdb_guerra_raw = df_refined[df_refined['genres'].str.contains("'name': 'Guerra'", na=False)]
print("🎯 Total de filmes com 'Guerra' no campo bruto 'genres':", tmdb_guerra_raw['id_filme'].nunique())


🎯 Total de filmes com 'Guerra' no campo bruto 'genres': 0


In [53]:
print("Exemplo de tipo:", type(df_refined.loc[0, 'genres']))
print("Valor:", df_refined.loc[0, 'genres'])


Exemplo de tipo: <class 'list'>
Valor: [{'id': 12, 'name': 'Aventura'}, {'id': 10752, 'name': 'Guerra'}, {'id': 36, 'name': 'História'}]


In [54]:
# Mapeamento traduzido
mapeamento_generos = {
    'Action': 'Ação',
    'Adventure': 'Aventura',
    'Animation': 'Animação',
    'Comedy': 'Comédia',
    'Crime': 'Crime',
    'Documentary': 'Documentário',
    'Drama': 'Drama',
    'Family': 'Família',
    'Fantasy': 'Fantasia',
    'History': 'História',
    'Horror': 'Terror',
    'Music': 'Música',
    'Mystery': 'Mistério',
    'Romance': 'Romance',
    'Science Fiction': 'Ficção científica',
    'TV Movie': 'Filme de TV',
    'Thriller': 'Suspense',
    'War': 'Guerra',
    'Western': 'Faroeste'
}

# Função robusta para extrair e traduzir os gêneros
def reconstruir_generos_traduzido(row):
    try:
        generos_traduzidos = []

        # Caso 1: genres já é lista (vindo da API TMDB tratada)
        if isinstance(row['genres'], list):
            generos_traduzidos = [mapeamento_generos.get(g['name'], g['name']) for g in row['genres'] if 'name' in g]

        # Caso 2: genres é string parecida com lista
        elif isinstance(row['genres'], str) and row['genres'].startswith('['):
            import ast
            lista = ast.literal_eval(row['genres'])
            generos_traduzidos = [mapeamento_generos.get(g['name'], g['name']) for g in lista if 'name' in g]

        # Caso 3: usar 'genero' do CSV como fallback
        elif pd.notnull(row.get('genero')):
            return row['genero']

        return ','.join(generos_traduzidos) if generos_traduzidos else pd.NA

    except Exception:
        return pd.NA

# Aplicar ao DataFrame
df_refined['generos'] = df_refined.apply(reconstruir_generos_traduzido, axis=1)


In [55]:
qtd_war = df_refined[df_refined['generos'].str.contains('Guerra', na=False)].shape[0]
print(f"🎯 Total de filmes com gênero 'Guerra' agora corretamente identificados: {qtd_war}")


🎯 Total de filmes com gênero 'Guerra' agora corretamente identificados: 1942


entendi o erro e vou voltar a tratara a tabela fato

3. Código para extrair e padronizar os gêneros em ambos:

In [57]:
import pandas as pd
import ast

# Exemplo carregando os CSVs (supondo pipe para movies.csv)
df_movies = pd.read_csv('movies.csv', sep='|', na_values='\\N')
df_tmdb = pd.read_csv('filmes_guerra_detalhes.csv')

# Função para extrair gênero do movies.csv (string simples)
def extrair_generos_csv(gen):
    if pd.isna(gen):
        return []
    # separa por vírgula e remove espaços
    return [g.strip() for g in gen.split(',')]

# Função para extrair gênero do tmdb.csv (string json de lista de dicts)
def extrair_generos_tmdb(gen_str):
    try:
        # converte string para lista de dicts
        gen_list = ast.literal_eval(gen_str)
        # extrai os nomes dos gêneros
        return [g['name'] for g in gen_list]
    except:
        return []

# Criar coluna genres_padronizado nos dois dfs
df_movies['genres_padronizado'] = df_movies['genero'].apply(extrair_generos_csv)
df_tmdb['genres_padronizado'] = df_tmdb['genres'].apply(extrair_generos_tmdb)

# Exemplo: filtrar filmes de guerra (gênero "Guerra")
filmes_guerra_movies = df_movies[df_movies['genres_padronizado'].apply(lambda x: 'Guerra' in x)]
filmes_guerra_tmdb = df_tmdb[df_tmdb['genres_padronizado'].apply(lambda x: 'Guerra' in x)]

print(f"Filmes de guerra no CSV movies: {len(filmes_guerra_movies)}")
print(f"Filmes de guerra no CSV TMDB: {len(filmes_guerra_tmdb)}")


Filmes de guerra no CSV movies: 0
Filmes de guerra no CSV TMDB: 574


In [58]:
print(df_movies['genero'].value_counts(dropna=False))


genero
Drama                               209429
Comedy                              103839
Comedy,Drama                         39051
NaN                                  38880
Drama,Romance                        38132
Horror                               29096
Comedy,Romance                       23088
Documentary                          18476
Action                               18235
Comedy,Drama,Romance                 17935
Thriller                             17857
Crime,Drama                          16099
Western                              13911
Drama,Thriller                       12529
Action,Crime,Drama                   12397
Romance                              11919
Action,Drama                         11783
Horror,Thriller                       9415
Drama,War                             8443
Crime,Drama,Thriller                  7744
Adventure                             7665
Drama,Family                          7628
Crime                                 6475
Come

In [59]:
# Importante: ignora maiúsculas/minúsculas para garantir mais abrangência
filtro_movies_guerra = df_movies['genero'].str.contains('War', case=False, na=False)

# Quantidade total de filmes com 'War' no genero no movies.csv
print(f"🎯 Total de filmes de guerra no CSV movies: {filtro_movies_guerra.sum()}")


🎯 Total de filmes de guerra no CSV movies: 22689
